In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
from math import ceil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from fbprophet import Prophet

In [3]:
df = pd.read_csv("csv/총 분석.csv")
df_1=pd.read_csv('데이터/01_제공데이터_update(210811)/01_음식물쓰레기_FOOD_WASTE_210811_update.CSV',low_memory=False,encoding='euc-kr')
예상데이터=pd.read_csv('csv/시계열총데이터최종본.CSV',encoding='euc-kr')

## * 실수형을 소수점 둘째자리까지 표현

In [3]:
pd.options.display.float_format='{:.2f}'.format

## * Random Forest로 학습

In [4]:
from sklearn.ensemble import RandomForestRegressor

data_y=df['em_g']
X_features=df.drop(['em_g','emd_nm','base_date','resid_reg_pop','foreign_pop','total_pop'],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X_features,data_y,test_size=0.3,random_state=777)
scaler = StandardScaler()
scaled_val = scaler.fit_transform(X_features)
scaled_x = pd.DataFrame(scaled_val, columns=X_features.columns)
model_randomForest = RandomForestRegressor() # 생성할 트리 개수가 얼만큼이 좋은지 모르겠음
model_randomForest.fit(X_train, y_train.values.ravel())

RandomForestRegressor()

In [5]:
model_randomForest.score(X_test,y_test)

0.9713929744942831

## * 예상데이터에서 사용할 7,8월 데이터를 저장

In [6]:
예상7월=예상데이터[['내국인7R','내국인7W','내국인7V','내국인관광7W','내국인관광7V','장기외국인7R','장기외국인7W','장기외국인7V',
          '단기외국인7V','간식7','농축수산물7','마트/슈퍼마켓7','배달7','부페7','식품7','아시아음식7','양식7','주점및주류판매7',
          '패스트푸드7','한식7']]
예상7월.columns=['내국인R', '내국인W', '내국인V', '내국인관광W','내국인관광V', '장기외국인R', '장기외국인W', '장기외국인V', '단기외국인V', '간식', '농축수산물', '마트/슈퍼마켓', '배달', '부페', '식품','아시아음식', '양식', '주점및주류판매', '패스트푸드', '한식']

In [7]:
예상8월=예상데이터[['내국인8R','내국인8W','내국인8V','내국인관광8W','내국인관광8V','장기외국인8R','장기외국인8W','장기외국인8V',
          '단기외국인8V','간식8','농축수산물8','마트/슈퍼마켓8','배달8','부페8','식품8','아시아음식8','양식8','주점및주류판매8',
          '패스트푸드8','한식8']]
예상8월.columns=['내국인R', '내국인W', '내국인V', '내국인관광W','내국인관광V', '장기외국인R', '장기외국인W', '장기외국인V', '단기외국인V', '간식', '농축수산물', '마트/슈퍼마켓', '배달', '부페', '식품','아시아음식', '양식', '주점및주류판매', '패스트푸드', '한식']

## * 위에서 학습한 모델로 7,8월 예측

In [8]:
predict7=model_randomForest.predict(예상7월)
predict8=model_randomForest.predict(예상8월)

In [9]:
마을list=sorted(df['emd_nm'].unique())

In [10]:
predict7=pd.DataFrame(predict7)
predict8=pd.DataFrame(predict8)
predict7.index=마을list
predict8.index=마을list

In [11]:
predict7.columns=['7월예측 em_g']

In [12]:
predict8.columns=['8월예측 em_g']

In [13]:
예상7_8월=pd.concat([predict7,predict8],axis=1)

## * 알수없음 같은경우는 시계열 예측으로 7,8월 쓰레기양 예측

In [14]:
알수없음쓰레기양=df_1[df_1['emd_nm']=='알수없음']
알수없음쓰레기양=알수없음쓰레기양.drop(['city','emd_cd','emd_nm','em_cnt','pay_amt','em_area_cd'],axis=1)
알수없음쓰레기양=알수없음쓰레기양.reset_index()
알수없음쓰레기양=알수없음쓰레기양.drop('index',axis=1)

In [15]:
알수없음쓰레기양.columns=['ds','y']
알수없음쓰레기양
m=Prophet(daily_seasonality=True)
m.fit(알수없음쓰레기양)
future=m.make_future_dataframe(periods=62)
pre_future=m.predict(future)

INFO:numexpr.utils:NumExpr defaulting to 6 threads.


In [16]:
쓰레기예측량=pre_future[['ds','yhat']].tail(62)
쓰레기예측량=쓰레기예측량.reset_index()
쓰레기7월예측량=쓰레기예측량[(쓰레기예측량['index']<944)]
쓰레기8월예측량=쓰레기예측량[(쓰레기예측량['index']>=944)]
쓰레기7월예측량=pd.DataFrame(쓰레기7월예측량)
쓰레기8월예측량=pd.DataFrame(쓰레기8월예측량)

In [17]:
쓰레기7월예측량=쓰레기7월예측량.drop(['index','ds'],axis=1)
쓰레기8월예측량=쓰레기8월예측량.drop(['index','ds'],axis=1)

In [18]:
쓰레기7월예측량=쓰레기7월예측량.sum()
쓰레기8월예측량=쓰레기8월예측량.sum()

## * 시계열로 예측한 알수없음 쓰레기양도 7,8월 예측 데이터 셋에 삽입

In [19]:
쓰레기7월예측량=pd.DataFrame(쓰레기7월예측량)
쓰레기8월예측량=pd.DataFrame(쓰레기8월예측량)

In [20]:
쓰레기7월예측량.columns=['7월예측 em_g']
쓰레기7월예측량.index=['알수없음']
쓰레기8월예측량.columns=['8월예측 em_g']
쓰레기8월예측량.index=['알수없음']

In [21]:
예상7월=pd.concat([predict7,쓰레기7월예측량])
예상8월=pd.concat([predict8,쓰레기8월예측량])